# Demo: Analysis

This example is the continuation of the previous example: [filling gaps](https://metobs-toolkit.readthedocs.io/en/latest/examples/filling_example.html). This example serves as an introduction to the Analysis module.

In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None} #else the legend is cutoff in ipython inline plots

In [ ]:

import metobs_toolkit

your_dataset = metobs_toolkit.Dataset()
your_dataset.update_file_paths(
    input_data_file=metobs_toolkit.demo_datafile, # path to the data file
    input_metadata_file=metobs_toolkit.demo_metadatafile,
    template_file=metobs_toolkit.demo_template,
)
#Import the data
your_dataset.import_data_from_file()

#Get LCZ, and landcover fractions will be used later on
your_dataset.get_LCZ()
your_dataset.get_landcover()

#Coarsen to 15-minutes frequencies
your_dataset.coarsen_time_resolution(freq='15T')

#Apply default quality control
your_dataset.apply_quality_control(obstype='temp') #we use the default settings in this example

#Interpret the outliers gaps.
your_dataset.convert_outliers_to_gaps()

#Fill gaps 
your_dataset.interpolate_gaps(obstype='temp')



## Creating an Analysis

The built-in analysis functionality is centered around the [*Analysis*](https://metobs-toolkit.readthedocs.io/en/latest/reference/analysis.html) class. First, create an *Analysis* object using the ``get_analysis()`` method.

In [ ]:
analysis = metobs_toolkit.Analysis(orig_dataset=your_dataset, 
                                   use_gapfilled_values=True)
analysis

## Analysis methods

An overview of the available analysis methods can be seen in the documentation of the ``Analysis`` class. The relevant methods depend on your data and your interests. As an example, a demonstration of the filter and diurnal cycle of the demo data.

### Filtering data

It is common to filter your data according to specific meteorological phenomena or periods in time. To do this you can use the ``apply_filter()`` method.

In [ ]:
#filter to non-windy afternoons in the Autumn.
subset = analysis.apply_filter('wind_speed <= 2.5 & season=="autumn" & hour > 12 & hour < 20')

subset.make_plot()


## Diurnal cycle 

To make a diurnal cycle plot of your Analysis use the ``get_diurnal_statistics()`` method:

In [ ]:
dirunal_statistics = subset.get_diurnal_statistics(colorby='name',
                                                   obstype='humidity', 
                                                   plot=True,
                                                   errorbands=True,
                                                  )
#Note that in this example statistics are computed for a short period and only for the non-windy autumn afternoons.

## Landcover correlation heatmap

If you are interested in the correlation matrices for landcoverfractions and an observationtype, you can use the ``Aanalysis.get_lc_correlation_matrices()`` method to do so. You can specify aggregation groups to compute correlations more specifically. The ``Analysis.plot_correlation_heatmap()`` method can be used to plot a landcover correlation matrix as a heatmap.

**Note**: The ``Aanalysis.get_lc_correlation_matrices()`` method is a complex method, and it is advised to read the documentation carefully before using it.

In [ ]:
dict_of_cor_matrices = analysis.get_lc_correlation_matrices(
                            obstype=["temp"],
                            groupby_labels=['hour', 'season']
                                )

# plot heatmap
analysis.plot_correlation_heatmap(
                            groupby_value=(4, 'autumn')) #plot LC correlation matrix for autumn temp records at 4h00.
